<a href="https://colab.research.google.com/github/DaeSeokSong/MachineLearningModels/blob/main/Prediction_StockPrice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Import Pakage

In [ ]:
%cd drive/My Drive/DeepLearning/BlackUp/

# AI 학습용
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential

# 기타
import lstm, time

# Funtion

In [ ]:
def Denomalization(dataset) :
    Denormalized_data = []
    for data in dataset :
        Denormalized_window = [float(window[0]) * (float(p) + 1) for p in data]
        Denormalized_data.append(Denormalized_window)

    result = np.array(Denormalized_data)
    return result

# Load Dataset

In [ ]:
"""
10년치 삼성전자 데이터 가져오기

filename = '10y_dataset_005930.csv'
data = pd.read_csv(filename)
"""

# s%p 500 지수 데이터 불러오기, 국제 신용평가기관(미국의 스탠다드 푸어스)가 작성한 주가 지수 = 500개 대형기업(미국)의 주식을 포함한 지수
X_train, y_train, X_test, y_test = lstm.load_data('sp500.csv', 50, True)

# Normalize(lstm 모듈에서 load_data할 때 정규화 적용됌)

In [ ]:
"""
# 정규화를 안 하면 모델에 fit(학습)시에 loss율이 심해진다.
Normalized_data = []
for window in result :
    Normalized_window = [(float(p) / float(window[0]) - 1) for p in window]
    Normalized_data.append(Normalized_window)

result = np.array(Normalized_data)

# 테스트셋 / 훈련셋 분류
row = int(round(result.shape[0] * 0.9))
# 훈련셋 분류 및 랜덤 셔플
train = result[:row, :]
np.random.shuffle(train)

x_train = train[:, :-1] # 총 51개의 윈도우 중 50개
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
y_train = train[:, -1] # 총 51개의 윈도우 중 1개

x_test = result[row:, :-1] # 총 51개의 윈도우 중 50개
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
y_test = result[row:, -1] # 총 51개의 윈도우 중 1개
"""

# Model

In [ ]:
"""
셀 스테이트(cell state) = 내부에 있는 게이트(gate)들을 통해 LSTM에서 어떤 정보를 기억하고 어떤 정보를 버릴지에 대해 추가적인 학습을 가능하게 한다. 
이를 통해 RNN이 가진, 기울기 소실 문제(Vanishing Gradient Problem, 역전파를 통해 학습할 때 학습 기울기가 너무 작아져 학습이 잘 안 되는 문제)를 해결할 수 있다.
"""
# keras.models 중 모델을 순차적으로 정의하는 클래스
model = Sequential()

model.add(LSTM(
    input_dim=1, # input_dim == 입력 데이터의 차원
    output_dim=50, # output_dim == 결과 데이터의 차원
    return_sequences=True)) # return_sequences == 각 시퀀스를 출력할지
model.add(Dropout(0.2)) # 과적합 방지용 Dropout 20%(==0.2)

model.add(LSTM(
    100, # 노드 100개
    return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(output_dim=1))
model.add(Activation("linear")) # 활성화(Activation)함수 = linear

model.summary()

# Training

In [ ]:
# trainSet이나 testSet에 null값 섞여있으면 loss(손실률)이 nan으로 나오니 주의
model.fit(X_train, y_train, # 훈련셋
          batch_size=512, # 한 번에 몇 개씩 묶어서 학습시킬지, batch_size
          epochs=1, # N번 동안 반복해라, epochs
          validation_split=0.05) # 주어진 훈련셋에서 몇 퍼 만큼 평가셋으로 쓸지에 대한 비율

# Prediction

In [ ]:
# 모델 예측
pred = lstm.predict_sequences_multiple(model, X_test, 50, 50)
lstm.plot_results_multiple(pred, y_test, 50)

# CopyRight

In [ ]:
# 참조 사이트: https://github.com/llSourcell/How-to-Predict-Stock-Prices-Easily-Demo